In [6]:
from pathlib import Path
import zipfile
import requests

# 1. Get data

my_data_path = Path('my_data')

images_path = my_data_path / 'my_pizza_steak_sushi'
zip_name = 'my_pizza_steak_sushi.zip'

if images_path.is_dir():
    print(f'{images_path} exists')
else:
    print(f'Can\'t find {images_path}, I will create one ...')
    images_path.mkdir(parents=True, exist_ok=True)

    with open(my_data_path / zip_name, 'wb') as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print('downloading files ...')
        f.write(request.content)

    with zipfile.ZipFile(my_data_path / zip_name) as f:
        print('Unzippping data ...')
        f.extractall(images_path)

my_data\my_pizza_steak_sushi exists


In [7]:
# 2. Become one with the data
import os
def walk_through_dir(dir_path):
  """Walks through dir_path returning file counts of its contents."""
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(images_path)

There are 2 directories and 0 images in 'my_data\my_pizza_steak_sushi'.
There are 3 directories and 0 images in 'my_data\my_pizza_steak_sushi\test'.
There are 0 directories and 25 images in 'my_data\my_pizza_steak_sushi\test\pizza'.
There are 0 directories and 19 images in 'my_data\my_pizza_steak_sushi\test\steak'.
There are 0 directories and 31 images in 'my_data\my_pizza_steak_sushi\test\sushi'.
There are 3 directories and 0 images in 'my_data\my_pizza_steak_sushi\train'.
There are 0 directories and 78 images in 'my_data\my_pizza_steak_sushi\train\pizza'.
There are 0 directories and 75 images in 'my_data\my_pizza_steak_sushi\train\steak'.
There are 0 directories and 72 images in 'my_data\my_pizza_steak_sushi\train\sushi'.


In [8]:
# Setup train and testing paths

train_path = images_path / 'train'
test_path = images_path / 'test'

train_path, test_path

(WindowsPath('my_data/my_pizza_steak_sushi/train'),
 WindowsPath('my_data/my_pizza_steak_sushi/test'))

In [9]:
# Write transform for turning images into tensors

from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

In [10]:
# Use ImageFolder to create dataset(s)
from torchvision import datasets

train_data = datasets.ImageFolder(train_path, data_transform, target_transform=None)
test_data = datasets.ImageFolder(test_path, data_transform)

In [11]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=1, num_workers=0, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, num_workers=0, shuffle=False)

In [13]:
import torch
from torch import nn

input_shape = 3
hidden_units = 10
x, y = next(iter(train_dataloader))

conv_block_1 = nn.Sequential(
    nn.Conv2d(in_channels=input_shape,
              out_channels=hidden_units,
              kernel_size=3,  # how big is the square that's going over the image?
              stride=1,  # default
              padding=1),  # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
    nn.ReLU(),
    nn.Conv2d(in_channels=hidden_units,
              out_channels=hidden_units,
              kernel_size=3,
              stride=1,
              padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,
                 stride=2)  # default stride value is same as kernel_size
)

conv1 = nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
relu = nn.ReLU()
max_pool1 = nn.MaxPool2d(kernel_size=1) 

def print_shape(val, name):
    print(f'{name}.shape: {val.shape}')

print_shape(x, 'x')

conv1_res = conv1(x)
print_shape(conv1_res, 'conv1(x)')

relu1_res = relu(conv1_res)
print_shape(relu1_res, 'relu(conv1(x))')

conv2_res = conv2(relu1_res)
print_shape(conv2_res, 'conv2(relu(conv1(x)))')

max_pool1_res = max_pool1(conv2_res)
print_shape(max_pool1_res, 'max_pool1(conv2(relu(conv1(x))))')

x.shape: torch.Size([1, 3, 64, 64])
conv1(x).shape: torch.Size([1, 10, 64, 64])
relu(conv1(x)).shape: torch.Size([1, 10, 64, 64])
conv2(relu(conv1(x))).shape: torch.Size([1, 10, 64, 64])
max_pool1(conv2(relu(conv1(x)))).shape: torch.Size([1, 10, 64, 64])
